In [148]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import SGD,RMSprop
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers.normalization import BatchNormalization

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

seed = 26

import warnings
warnings.filterwarnings('ignore')

In [182]:
train_X = pd.read_csv('train_250.csv')
test_X = pd.read_csv('test_250.csv')
train_y = pd.read_csv('y_train.csv')

In [183]:
print("Train set size: {}".format(train_X.shape))
print("Test set size: {}".format(test_X.shape))

Train set size: (4459, 250)
Test set size: (49342, 250)


In [17]:
train_X.head()

,leak,log_leak,Var,Max,Std,SumZeros,weight_count,SumValues,srp_6,grp_1,...,400e9303d,e94c03517,64e38e7a2,cbf236577,203c64df6,9a9fc1aba,bdf773176,129fe0263,190db8488,c5dacc85b
0,4.220134,0.966739,-0.122376,-0.204619,-0.228293,0.259388,-0.174264,-0.259388,3.443512,-6.549093,...,-0.188571,-0.077914,-0.108463,-0.094536,-0.061265,-0.105151,1.162975,-0.119694,-0.265703,-0.099411
1,-0.536472,0.256984,-0.125737,-0.164270,-0.275031,0.424440,-0.366371,-0.424440,1.430268,3.883112,...,-0.188571,-0.077914,-0.108463,-0.094536,-0.061265,-0.105151,-0.150764,-0.119694,-0.265703,-0.099411
2,-0.612781,-2.019324,-0.136551,-0.317599,-0.539275,0.655512,-0.483598,-0.655512,1.458660,1.533526,...,-0.188571,-0.077914,-0.108463,-0.094536,-0.061265,-0.105151,-0.150764,-0.119694,-0.265703,-0.099411
3,-0.358417,0.462973,-0.136937,-0.341809,-0.574045,0.636649,-0.530309,-0.636649,1.459675,-1.008095,...,-0.188571,-0.077914,-0.108463,-0.094536,-0.061265,-0.105151,-0.150764,-0.119694,-0.265703,-0.099411
4,-0.612781,-2.019324,-0.133592,-0.214053,-0.425246,0.617786,-0.390725,-0.617786,1.450892,-2.609483,...,-0.188571,-0.077914,-0.108463,-0.094536,-0.061265,-0.105151,-0.150764,-0.119694,-0.265703,-0.099411


# Standardize Data

In [136]:
#Already has standard scaling applied in feature engineering
train_X.shape[1]

500

# Build Keras NN Model

# Crossfold analysis

In [144]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    
    
    model.add(Dense(200, input_dim=train_X.shape[1], kernel_initializer='normal', activation='relu'))

    model.add(Dense(200, kernel_initializer='normal', activation='relu'))
    
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))

    
    model.add(Dense(1, kernel_initializer='normal'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam') #sgd
    
    return model

In [145]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=10, verbose=1)

In [146]:
kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(estimator, 
                          train_X, 
                          train_y, 
                          cv=kfold, scoring="mean_squared_error")

print("Results: %.2f (%.2f) RMSE" % (np.sqrt(-results.mean()), results.std()))

Epoch 1/50
2229/2229 [==============================] - 3s 1ms/step - loss: 28.5833
Epoch 2/50
2229/2229 [==============================] - 1s 536us/step - loss: 6.6689
Epoch 3/50
2229/2229 [==============================] - 1s 569us/step - loss: 3.6704
Epoch 4/50
2229/2229 [==============================] - 1s 607us/step - loss: 2.4504
Epoch 5/50
2229/2229 [==============================] - 1s 515us/step - loss: 2.0045
Epoch 6/50
2229/2229 [==============================] - 1s 562us/step - loss: 2.1180
Epoch 7/50
2229/2229 [==============================] - 1s 576us/step - loss: 2.1431
Epoch 8/50
2229/2229 [==============================] - 1s 518us/step - loss: 1.7981
Epoch 9/50
2229/2229 [==============================] - 1s 535us/step - loss: 1.7788
Epoch 10/50
2229/2229 [==============================] - 1s 611us/step - loss: 1.9015
Epoch 11/50
2229/2229 [==============================] - 1s 527us/step - loss: 1.6467
Epoch 12/50
2229/2229 [==============================] - 1s 544u

2230/2230 [==============================] - 1s 526us/step - loss: 0.6121
Epoch 47/50
2230/2230 [==============================] - 1s 550us/step - loss: 0.6962
Epoch 48/50
2230/2230 [==============================] - 1s 538us/step - loss: 0.6081
Epoch 49/50
2230/2230 [==============================] - 1s 591us/step - loss: 0.5783
Epoch 50/50
2229/2229 [==============================] - 1s 621us/step
Results: 1.94 (0.26) RMSE


# Train/Val Analysis

NEED TO BEAT 0.61

@ 2 fold cross val, 50 epochs, 32 batch size, adam, relu
Baseline - 2.4 RMSE
Wide 1500 - 3.22 RMSE
Deep 750 -> 300 - 2.42 RMSE
Deep 750 -> 200 - 2.33 RMSE
Deep 750 -> 100 - 2.42 RMSE
Deep 750 -> 150 - 2.37 RMSE


750 ->200/200 - 2.38
750 ->200/200/200 - 2.31
750 ->200/200/200/200 - 2.27
750 ->200/200/200/200 200/200/200/200- 2.17

all_features -> 200 - 3.62 @ 10 epochs SLOW AF

500 -> 200 - 2.26 @ batch size 32
500 -> 200 -  2.20 @ batch size 25
500 -> 200 -  2.11 @ batch size 15
500 -> 200 -  2.00 @ batch size 10

500 -> 200 -  2.06 @ batch size 10 w/ batchnormalization

@batch size 10, 50 epochs
200 -> 200 - 1.99
200 -> 200 -> 50 - 1.94
200 * 8 - 1.5
200 * 8 -> 50 - 1.3
200 * 16 -> 50 - 1.29
200 * 8 -> 100 -> 50 -> 10 - 1.96

200 * 8 -> 10 - 1.75

200 * 8 -> 100 - 1.2

200 * 32 -> 100 - 1.18

200 * 8 -> 100 - 1.17 (250 features)






In [186]:
X_tr, X_val, y_tr, y_val = train_test_split(train_X, train_y)

In [187]:
model = Sequential()
    
model.add(Dense(200, input_dim=train_X.shape[1], kernel_initializer='normal', activation='relu'))

for i in range(1,8):
    model.add(Dense(200, kernel_initializer='normal', activation='relu'))

model.add(Dense(100, kernel_initializer='normal', activation='relu'))

model.add(Dense(1, kernel_initializer='normal'))
    
model.compile(loss='mean_squared_error', optimizer='adam') #sgd

In [188]:
model.fit(X_tr,y_tr,validation_data=(X_val,y_val),nb_epoch=50,batch_size=10)

Train on 3344 samples, validate on 1115 samples
Epoch 1/50
3344/3344 [==============================] - 13s 4ms/step - loss: 32.8891 - val_loss: 8.8963
Epoch 2/50
3344/3344 [==============================] - 7s 2ms/step - loss: 7.0352 - val_loss: 5.6772
Epoch 3/50
3344/3344 [==============================] - 8s 2ms/step - loss: 3.2924 - val_loss: 3.6004
Epoch 4/50
3344/3344 [==============================] - 7s 2ms/step - loss: 2.5277 - val_loss: 4.6638
Epoch 5/50
3344/3344 [==============================] - 10s 3ms/step - loss: 2.1583 - val_loss: 2.5523
Epoch 6/50
3344/3344 [==============================] - 9s 3ms/step - loss: 1.7342 - val_loss: 4.9850
Epoch 7/50
3344/3344 [==============================] - 10s 3ms/step - loss: 1.7605 - val_loss: 2.1183
Epoch 8/50
3344/3344 [==============================] - 10s 3ms/step - loss: 1.4497 - val_loss: 1.8895
Epoch 9/50
3344/3344 [==============================] - 8s 2ms/step - loss: 1.4574 - val_loss: 1.8876
Epoch 10/50
3344/3344 [======

In [189]:
np.sqrt(model.evaluate(X_val,y_val))

1115/1115 [==============================] - 0s 305us/step


1.1760197716709695

In [ ]:
def gen_data() :
X, Y = [], []
for x in xrange(1000) :
X.append(np.random.random_sample(5,))
Y.append(np.random.random_sample())
return np.asarray(X), np.asarray(Y)
def temp_data() :
X, Y = [], []
for x in xrange(1000) :
sample = []
for x in xrange(4) :
sample.append(np.random.random_sample(5,))
X.append(sample)
Y.append(np.random.random_sample())
return np.asarray(X), np.asarray(Y)
if name == 'main':
X,Y = temp_data()
model = Sequential()
model.add(LSTM(5, 10, activation='sigmoid', inner_activation='hard_sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(10,1, init='uniform', activation='linear'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)